In [ ]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
import sklearn.ensemble as ens
import xgboost as xgb
import json
import os
import re
from sklearn import preprocessing

In [ ]:
def SentimentData(type):
    petIds = []
    magnitudes = []
    scores = []
    for sentimentFilename in os.listdir('../input/petfinder-adoption-prediction/'+type+'_sentiment'):
        if re.search(r'.*\.json', sentimentFilename):
            f = open('../input/petfinder-adoption-prediction/'+type+'_sentiment/'+ sentimentFilename, 'r')
            jsonContent = json.loads(f.read())
            magnitude = jsonContent['documentSentiment']['magnitude']
            score = jsonContent['documentSentiment']['score']
            petIds.append(sentimentFilename.split('.')[0])
            magnitudes.append(float(magnitude))
            scores.append(float(score))

    rows = []
    for i in range(len(petIds)):
        rows.append([petIds[i], magnitudes[i], scores[i]])
    columns = ['PetID', 'SentimentMagnitude', 'SentimentScore']

    sentimentDf=pd.DataFrame(rows)
    sentimentDf.columns = columns
    return sentimentDf

In [ ]:
def SentimentData1(data):
    sentiment = 
    if re.search(r'.*\.json', data['PetID']+'.json'):
        f = open('../input/petfinder-adoption-prediction/train_sentiment/'+data['PetID']+'.json', 'r')
        jsonContent = json.loads(f.read())
        jsonContent['sentences'][0]['sentiment']['magnitude']
        jsonContent['sentences'][0]['sentiment']['score']
# jsonContent

In [ ]:
train_data = pd.read_csv(r'../input/petfinder-adoption-prediction/train/train.csv')
test_data = pd.read_csv(r'../input/petfinder-adoption-prediction/test/test.csv')

sentimentDf = SentimentData('train')
train = pd.DataFrame()
train = pd.merge(train_data, sentimentDf, how="left", on='PetID')

sentimentTestDf = SentimentData('test')
test = pd.DataFrame()
test = pd.merge(test_data, sentimentTestDf, how="left", on='PetID')
train['SentimentScore'].iloc[train['SentimentScore'].isna()]=0
train['SentimentMagnitude'].iloc[train['SentimentMagnitude'].isna()]=0
test['SentimentScore'].iloc[test['SentimentScore'].isna()]=0
test['SentimentMagnitude'].iloc[test['SentimentMagnitude'].isna()]=0

In [ ]:
def preProcesser(data):
    data['IsPureBreed'] = (data["Breed1"] == 0) | (data["Breed2"] == 0) | (data["Breed1"] == data["Breed2"])
    
    data['HavePhoto'] = data['PhotoAmt']
    data.loc[data['HavePhoto']==0, 'HavePhoto'] = False
    data.loc[data['HavePhoto']!=0, 'HavePhoto'] = True
    data['PhotoAmt_Normalize'] = data['PhotoAmt']/data['PhotoAmt'].max()
    
    data['DescriptionLength'] = data['Description'].map(str).apply(len)
    data['DescriptionLength'] = data['DescriptionLength']/data['DescriptionLength'].max()
    
    le = preprocessing.LabelEncoder()
    le.fit(data['RescuerID'])
    data['EncodeRescuerID'] = le.transform(data['RescuerID'])
    
    temp = ((data['Color1']==0) & (data['Color2']==0))|((data['Color1']==0) & (data['Color3']==0))|((data['Color3']==0) & (data['Color2']==0))
    data['NumberColor'] = [1 if (tmp == True) else 0 for tmp in temp]
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']==0))|((data['Color1']!=0) & (data['Color3']!=0)& (data['Color2']==0))|((data['Color3']!=0) & (data['Color2']!=0) & (data['Color1']==0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 2
    temp = ((data['Color1']!=0) & (data['Color2']!=0) & (data['Color3']!=0))
    data.loc[[tmp == True for tmp in temp], 'NumberColor'] = 3
    
    data['AgeYear'] = data['Age']//12
    
    data['onePet'] = (data['Quantity']==1)
    
    data['HaveVideo'] = (data['VideoAmt']!=0)
    
#     threshold1 = 0.6
#     threshold2 = - threshold1
#     thOfMagnitude = 3
#     Positive = 5
#     Negative = -5
#     Neutral = 0
#     Mixed = 2
#     temp = []
#     for s, m in zip(data['SentimentScore'],data['SentimentMagnitude']):
#         if s >= threshold1:
#             temp.append(Positive)
#         elif s > threshold2 and s < threshold1 and m >= thOfMagnitude:
#             temp.append(Mixed)
#         elif s > threshold2 and s < threshold1 and m < thOfMagnitude:
#             temp.append(Neutral)
#         elif s <= threshold2:
#             temp.append(Negative)
    data['Sentiment'] = data.apply(sentimet, axis=1)
    return data

def sentimet(data):
    return data['SentimentScore']*data['SentimentMagnitude']

In [ ]:
train_pre = preProcesser(train)
test_pre = preProcesser(test)

In [ ]:
train_image = train.copy()
train_image = train_image.reset_index()
def append(img):
    if img['HavePhoto']:
        return '../input/petfinder-adoption-prediction/train_images/'+img['PetID']+'-1.jpg'
    return ''
train_image['Input_image'] = train_image.apply(append, axis=1)

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
from keras.layers import GlobalAveragePooling2D, Input, Lambda, AveragePooling1D
import keras.backend as K
import numpy as np
from sklearn.decomposition import NMF
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

inp = Input((224,224,3))
model = VGG19(input_tensor = inp, weights='imagenet',include_top=False)
x = model.output
x = GlobalAveragePooling2D()(x)
x = Lambda(lambda x: K.expand_dims(x,axis = -1))(x)
x = AveragePooling1D(16)(x)
out = Lambda(lambda x: x[:,:,0])(x)
model = Model(inp,out)
# model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
def loadImage(data):
    if data['Input_image']!='':
        img = image.load_img(data['Input_image'], target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        a = model.predict(x)
        svd = NMF(n_components=32)
        a = svd.fit_transform(a)
        print(data['index'], end='\r')
        return a[0]
    return np.zeros(32)
model.summary()

In [ ]:
train_image['Image'] = train_image.apply(loadImage, axis=1)

In [ ]:
train_image1 = train_image.copy()
df3 = pd.DataFrame(train_image1['Image'].to_list(), columns=[i for i in range(32)])
train_image1=pd.concat([train_image1,df3], axis=1)

In [ ]:
dataWithSentiment_train = train_image1.copy()
dataWithSentiment_label = pd.DataFrame()
dataWithSentiment_label['AdoptionSpeed'] = dataWithSentiment_train['AdoptionSpeed']
dataWithSentiment_train = dataWithSentiment_train.drop(['AdoptionSpeed','Name','Description','RescuerID','PetID','Color1', 'Color2', 'Input_image','Image'], axis=1)
test1 = test_pre.drop(['Name','Description','RescuerID','PetID','Color1', 'Color2'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(dataWithSentiment_train, dataWithSentiment_label['AdoptionSpeed'], test_size=0.33, random_state=42)

In [ ]:
clf = RandomForestClassifier(max_depth=100,n_estimators = 1000,random_state=1000)
clf.fit(X_train.values, y_train)
print(clf.score(dataWithSentiment_train.values, dataWithSentiment_label['AdoptionSpeed']))
print(clf.score(X_test.values, y_test))

In [ ]:
# model = ens.GradientBoostingClassifier(max_depth= 10, n_estimators = 100,random_state=1000)
# model.fit(X_train.values, y_train)
# model.score(X_test.values, y_test)

In [ ]:
output = clf.predict(test1)

In [ ]:
sub = pd.DataFrame()
sub['PetID'] = test['PetID']
sub['AdoptionSpeed'] = output
sub.to_csv('submission.csv', index=False)